In [3]:
import pandas as pd
import googlemaps
import pickle
import os
import boto3
import time

In [4]:
s3 = boto3.client('s3')
BUCKET_NAME = 'capstonehaystacks'
FILE_NAME = 'points-of-interest-google2.csv'

file_obj = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_NAME)
POI_google = pd.read_csv(file_obj['Body'])

In [5]:
POI_google.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,zip_code,primary_category
0,H&R Block,"H&R Block, 802 Lower Fayetteville Rd Ste C, Ne...",0x88f4c6329748b5b7:0x412418d65ff2fcb7,NaN,33.374876,-84.762020,"['Tax preparation service', 'Tax consultant']",4.8,58,NaN,"{'Thursday': '10AM–5PM', 'Friday': 'Closed', '...","{'Health & safety': ['Mask required', 'Staff w...",Open ⋅ Closes 5PM,"['0x88f4c6685cd3510f:0x154ee5bc850164a6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30263,Tax preparation service
1,Gwinnett Community Bank,"Gwinnett Community Bank, 2775 Buford Hwy, Dulu...",0x88f5a28359cb48cd:0xacc5f8fde9522f87,NaN,34.007883,-84.133413,"['Bank', 'ATM']",3.0,2,NaN,"{'Thursday': '9AM–4PM', 'Friday': '9AM–5PM', '...","{'Service options': ['Drive-through'], 'Access...",Open ⋅ Closes 4PM,"['0x88f599224d1b5ec5:0xcf6966bb9d99d4fc', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30096,Bank
2,Corley Insurance Inc,"Corley Insurance Inc, 2855 Buford Hwy, Duluth,...",0x88f5a284e80502b3:0xfe92519aa603f4d9,NaN,34.006241,-84.136169,"['Insurance agency', 'Auto insurance agency', ...",4.1,8,NaN,"{'Thursday': '8:30AM–5:30PM', 'Friday': '8:30A...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 5:30PM,"['0x88f5a284fb58e8cd:0x8dd0cb71c88449ee', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30096,Insurance agency


In [20]:
with open('../data/secrets.pkl', 'rb') as f:
    secrets = pickle.load(f)


In [21]:
API_KEY = secrets['API_KEY']

In [22]:
gmaps = googlemaps.Client(key=API_KEY)

In [26]:
# testing out with 5 first rows
test_df = POI_google[:10]

In [27]:
# Define a function to retrieve the place type for a given place_id
def get_place_type(place_id):
    # Make a request to the Places API for the place details
    place_details = gmaps.place(place_id, fields=['type'])
    
    # Extract the place types from the response
    place_types = place_details['result'].get('type', [])
    
    # Return the first place type (if any)
    if len(place_types) > 0:
        return place_types[0]
    else:
        return None

# Define a function to retrieve the place type for each row in your dataframe
def get_place_types(df):
    # Create an empty list to store the place types
    place_types = []
    
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Retrieve the place type for the given place_id
        place_type = get_place_type(row['gmap_id'])
        
        # Append the place type to the list
        place_types.append(place_type)
        
        # Add a sleep to avoid exceeding API rate limits
        time.sleep(0.1)
    
    # Return the list of place types
    return place_types

In [28]:
# Call the function to retrieve the place types for each row
test_df['place_type'] = get_place_types(test_df)

ApiError: INVALID_REQUEST

In [31]:
# It looks like the error is caused by an "INVALID_REQUEST". 
# This can happen if the place_id passed to gmaps.place is not valid. 

get_place_type('0x88f4c6329748b5b7:0x412418d65ff2fcb7')

# Ok, the data frame PLace ID's aren't relavent anymore

ApiError: INVALID_REQUEST

In [30]:
# I found the ID for my local coffee shop and it works! so the ID's in the document are unusable
# https://developers.google.com/maps/documentation/places/web-service/place-id`

get_place_type('ChIJnXNTotvIwokRQK1m8aCCTgk')

# So we have a bigger issue that that dataframe can't be appended with anything from google API.